In [1]:
!pip install beautifulsoup4
!pip install requests

In [2]:
import requests
from bs4 import BeautifulSoup
import re

### Load data on IMDb by BeautifulSoup

In [3]:
# since movie title on IMDb would automatically convert my website title to chinese according to my web domain
#so I need to check out ther website for movie titles
url = "http://filmcentrum.nl/imdb250/"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
titles = soup.find_all('div',{'class':"lijstenFilm"})
years = soup.find_all("div", {'class': "lijstenJaar"})
process_title = []
count = 0
#print(titles)
# remove blank in titles
for i in titles:
    ii = i.get_text()
    process_title.append(ii[6:-2] + " ("+ str(years[count].get_text()) +")")
    count += 1
    
print(process_title)

['The Shawshank Redemption (1994)', 'The Godfather (1972)', 'The Godfather: Part II (1974)', 'The Dark Knight (2008)', '12 Angry Men (1957)', "Schindler's List (1993)", 'The Lord of the Rings: the Return of the King (2003)', 'Pulp Fiction (1994)', 'The Good, the Bad and the Ugly (1966)', 'The Lord of the Rings: the Fellowship of the Ring (2001)', 'Fight Club (1999)', 'Forrest Gump (1994)', 'Inception (2010)', 'Star Wars: Episode V - the Empire Strikes Back (1980)', 'The Lord of the Rings: the Two Towers (2002)', 'The Matrix (1999)', 'Goodfellas (1990)', "One Flew Over the Cuckoo's Nest (1975)", 'Seven Samurai (1954)', 'Se7en (1995)', 'City of God (2002)', 'La Vita è Bella (1997)', 'The Silence of the Lambs (1991)', "It's a Wonderful Life (1946)", 'Star Wars: Episode IV - a New Hope (1977)', 'Parasite (2019)', 'Saving Private Ryan (1998)', 'Spirited Away (2001)', 'The Green Mile (1999)', 'Interstellar (2014)', 'Léon (1994)', 'The Usual Suspects (1995)', 'Seppuku (1962)', 'The Lion King 

In [4]:
url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
links = soup.find_all("a", href = re.compile("^/title/tt\d"))
article = []
#link = links
for link in links[:200]:
    url = "https://www.imdb.com" + str(link['href']) + "plotsummary?ref_=tt_stry_pl#synopsis"
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    text = soup.find('li', id=re.compile('^synopsis-py'))
    # 'NoneType' object has no attribute 'get_text'
    if text is None: 
        pass
    else:
        article.append(text.get_text())
print(len(article))

KeyboardInterrupt: 

In [ ]:
import nltk,os
nltk.download('punkt')
doc_all = {}
i = 0
#print(article[0])
for title in process_title[:95]:
    tokens = nltk.word_tokenize(article[i])
    token_filtered = [w.lower() for w in tokens if w.isalpha()]
    doc_all[title]=token_filtered
    i += 1

### convert all document to vector, use tfidf weighting

In [ ]:
import numpy as np

def tf(term, token_doc):
    tf = token_doc.count(term)/len(token_doc)
    return tf

# create function to calculate how many doc contain the term 
def numDocsContaining(word, token_doclist):
    doccount = 0
    for doc_token in token_doclist:
        if doc_token.count(word) > 0:
            doccount +=1
    return doccount
  
import math
# create function to calculate  Inverse Document Frequency in doclist - this list of all documents
def idf(word, token_doclist):
    n = len(token_doclist)
    df = numDocsContaining(word, token_doclist)
    if df==0:
        return 0
    else:
        return math.log10(n/df)

#define a function to do cosine normalization a data dictionary
def cos_norm(dic): # dic is distionary data structure
    dic_norm={}
    factor=1.0/np.sqrt(sum([np.square(i) for i in dic.values()]))
    for k in dic:
        dic_norm[k] = dic[k]*factor
    return dic_norm

#create function to calculate normalize tfidf 
def compute_tfidf(token_doc,bag_words_idf):
    tfidf_doc={}
    for word in set(token_doc):
        if word not in bag_words_idf.keys(): # may not find keys
            pass
        else:
            tfidf_doc[word]= tf(word,token_doc) * bag_words_idf[word] 
    tfidf_norm = cos_norm(tfidf_doc)
    return tfidf_norm

# create normalize term frequency
def tf_norm(token_doc):
    tf_norm={}
    for term in token_doc:
        tf = token_doc.count(term)/len(token_doc)
        tf_norm[term]=tf
    tf_max = max(tf_norm.values())
    for term, value in tf_norm.items():
        tf_norm[term]= 0.5 + 0.5*value/tf_max
    return tf_norm

def compute_tfidf_query(query_token,bag_words_idf):
    tfidf_query={}
    tf_norm_query = tf_norm(query_token)
    for term, value in tf_norm_query.items():
        tfidf_query[term]=value*bag_words_idf[term]   
    return tfidf_query

In [ ]:
import os
from datetime import datetime

file = open("time_result.txt","w")#write mode 
now = datetime.now()
start = now.strftime("%H:%M:%S")
file.write("-------------------No feature selection--------------------")
print("Start tfidf processing at", start)
file.write("\nStart tfidf processing at "+ str(start))

#create bag words
bag_words =[] # declare bag_words is a list
for doc in doc_all.keys():
    bag_words += doc_all[doc]
bag_words=set(bag_words)

#calculate idf for every word in bag_words
bag_words_idf={}  
bag_words_len = len(bag_words)
bag_word_10 = round(bag_words_len/10,0)
print("the number of term in bag_word", bag_words_len)
print("Start idf processing")
i=0
for word in bag_words:
    i+=1
    if (i%bag_word_10==0):print("finish %s idf processing" %(str(round(i*10/bag_word_10))+"%"))
    bag_words_idf[word]= idf(word,doc_all.values())

print("finish idf processing")
##calculate tfidf with cosine normalization
tfidf={} # declare tfidf dictionary to store tfidf value
for doc in doc_all.keys():
    tfidf[doc]= compute_tfidf(doc_all[doc],bag_words_idf)
    #print(tfidf[doc])
print("finish tfidf processing")

### Input query text, and take similarity

In [ ]:
query="encourage wise hope"
query_token_raw= nltk.word_tokenize(query)
query_token = [term for term in query_token_raw if term in bag_words]

tfidf_query =compute_tfidf_query(query_token,bag_words_idf) #calculate tfidf for query text
print(tfidf_query)

In [ ]:
# add tfidf of query text to tfidf of all doc and convert to dataframe
tfidf["query"] = tfidf_query

import pandas as pd
tfidf_df = pd.DataFrame(tfidf).transpose()
tfidf_df= tfidf_df.fillna(0) # replace all NaN by zero

from scipy.spatial.distance import cosine
cosine_sim ={}
for row in tfidf_df.index:
    if row != "query":
        cosine_sim[row]= 1-cosine(tfidf_df.loc[row],tfidf_df.loc["query"])

# the top 10 relevant document
cosine_sim_top10 = dict(sorted(cosine_sim.items(), key=lambda item: item[1],reverse=True)[:10])
print(cosine_sim_top10)

now = datetime.now()
finish = now.strftime("%H:%M:%S")
print("Finish tfidf processing at", finish)
file.write("\nFinish tfidf processing at "+str(finish))
spent = finish - start
file.write("\nTotal spent time: "+str(spent))
file.close()

In [ ]:
#plot barchart
import matplotlib.pyplot as plt
data = cosine_sim_top10
plt.barh(range(len(data)), list(data.values()), align='center', alpha=0.8)
plt.yticks(range(len(data)), list(data.keys())) # label for y axis
plt.xlabel('Smimilarity score')
plt.ylabel('Course')

# save graph
path = r"C:\Users\USER\Desktop\Text-Mining\hw6"
plt.savefig(path+"\\barchart.png", bbox_inches='tight', dpi=600)
plt.show()

### With feature selection process

In [ ]:
from nltk.corpus import stopwords
from nltk import FreqDist
stopword = nltk.download('stopwords')
nltk.download('universal_tagset')

tokens_no_stop = [word for word in bag_words if not word in stopwords.words()]
pos_tokens = nltk.pos_tag(tokens_no_stop, tagset='universal')

In [ ]:
def extract_terms(tokens):
    NVA_tokens =[] 
    for token in tokens:
        if token[1] == "NOUN":
            NVA_tokens += [token[0]]
        elif token[1] == "VERB":
            NVA_tokens += [token[0]]
        elif token[1] == "ADJ":
            NVA_tokens += [token[0]]
    return NVA_tokens

new_tokens = extract_terms(pos_tokens)

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
movies_porter = []
for words in new_tokens:
    movies_porter.append(ps.stem(words))
    
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download("punkt")
nltk.download("maxent_treebank_pos_tagger")
nltk.download('averaged_perceptron_tagger')
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemmatize_sentence(words):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(words):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))

    return res

lem = WordNetLemmatizer()
lem_and_porter = lemmatize_sentence(movies_porter)

In [ ]:
freq_dist = FreqDist(lem_and_porter)
print(freq_dist.most_common(10))
print("Before extract:",len(bag_words))
print("After extract:",len(new_tokens))
print(len(lem_and_porter))

In [ ]:
#calculate idf for every word in bag_words
file = open("time_result.txt","a")#write mode 
now = datetime.now()
start = now.strftime("%H:%M:%S")
file.write("\n-------------------With feature selection--------------------")
print("Start tfidf processing at", start)
file.write("\nStart tfidf processing at "+ str(start))

new_tokens = set(lem_and_porter)
bag_words_idf={}  
bag_words_len = len(new_tokens)
bag_word_10 = round(bag_words_len/10,0)
print("the number of term after doing feature selection", bag_words_len)
print("Start idf processing")
i=0
for word in new_tokens:
    i+=1
    if (i%bag_word_10==0):print("finish %s idf processing" %(str(round(i*10/bag_word_10))+"%"))
    bag_words_idf[word]= idf(word,doc_all.values())

print("finish idf processing")
##calculate tfidf with cosine normalization
tfidf={} # declare tfidf dictionary to store tfidf value
for doc in doc_all.keys():
    tfidf[doc]= compute_tfidf(doc_all[doc],bag_words_idf)
    #print(tfidf[doc])
print("finish tfidf processing")

In [ ]:
tfidf["query"] = tfidf_query
tfidf_df = pd.DataFrame(tfidf).transpose()
tfidf_df= tfidf_df.fillna(0) # replace all NaN by zero

cosine_sim ={}
for row in tfidf_df.index:
    if row != "query":
        cosine_sim[row]= 1-cosine(tfidf_df.loc[row],tfidf_df.loc["query"])

# the top 10 relevant document
cosine_sim_top10 = dict(sorted(cosine_sim.items(), key=lambda item: item[1],reverse=True)[:10])
print(cosine_sim_top10)

now = datetime.now()
finish = now.strftime("%H:%M:%S")

print("\nFinish tfidf processing at", finish)
file.write("\nFinish tfidf processing at " + str(finish))
spent = finish - start
file.write("\nTotal spent time: "+str(spent))
file.close()

In [ ]:
data = cosine_sim_top10
plt.barh(range(len(data)), list(data.values()), align='center', alpha=0.8)
plt.yticks(range(len(data)), list(data.keys())) # label for y axis
plt.xlabel('Smimilarity score')
plt.ylabel('Course')

# save graph
path = r"C:\Users\USER\Desktop\Text-Mining\hw6"
plt.savefig(path+"\\barchart_with_Feature_Selection.png", bbox_inches='tight', dpi=600)
plt.show()